In [1]:
import pandas as pd
import requests
import time
import random
import requests
import json
from datetime import date
import os


In [4]:
def scrape_new():
    '''
    This function gets the json formatted post request result form the web API
    between certain time interval and scrapes the new pdf documents from the website and stores them
    in the ourput folder. This function returns the pandas dataframe of the meta-data of new documents.    
    '''
    # generate start and end date according to last record date of the documents:
    data = pd.read_json (r'output/responses_2018_now.json') # this will change as pickle file
    
    # The format of start and end time is:"2021-03-01T00:00:00.000Z"
    start = data.sort_values(by = "recordDate").iloc[-1]["recordDate"].split("+")[0]+"Z"
    end = str(date.today())+"T00:00:00.000Z"
    
    # convert json null and false to python None and False
    null = None
    false = False
    #API details
    url = "https://public-search.emploi.belgique.be/website-service/joint-work-convention/search"
    body = json.dumps({"lang":"fr",
            #"jc":"1200000",
            "title":"",
            "superTheme":"",
            "theme":"",
            "textSearchTerms":"",
            "signatureDate":{"start":null,"end":null},
            "depositNumber":{"start":49933,"end":null},
            "noticeDepositMBDate":{"start":null,"end":null},
            "enforced":"",
            "royalDecreeDate":{"start":null,"end":null},
            "publicationRoyalDecreeDate":{"start":null,"end":null},
            "recordDate":{"start":start,"end":end},
            "correctedDate":{"start":null,"end":null},
            "depositDate":{"start":null,"end":null},
            "advancedSearch":false})
    headers = {'Content-Type': 'application/json'}

    #Making http post request
    response = requests.post(url, headers=headers, data=body, verify=False)
    
    # keep the records as json file
    json_file = json.dumps(response.json())
    
    df = pd.read_json(json_file)
    base_url = r'https://public-search.emploi.belgique.be/website-download-service/joint-work-convention/'
    
    counter = 0
    save_path = "output/"
    # fetches the pdf file
    for url in df['documentLink']:
        r = requests.get(base_url + url, allow_redirects=True)
        # replacing / with - in the filename of the pdf
        file_name = url.replace('/', '-')
        complete_name = os.path.join(save_path, file_name)
        file = open(complete_name , 'wb')
        file.write(r.content)
        counter += 1
        #print (counter)
        # puts the thread randomly to sleep from 0.1 to 1.9 seconds
        random_number = random.randint(0,10) /10 + random.randint(0,1)
        time.sleep(random_number)    
        file.close()
    print (f"{counter} new document(s) added to output folder")
    #new_data_df = pd.json_normalize(response.json())
    return df
        

In [5]:
new_df = scrape_new()

/Users/mselmaesen/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-search.emploi.belgique.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


79 new document(s) added to output folder


In [6]:
new_df.head()

jcId                                             jcFr  \
0  1120000   COMMISSION PARITAIRE DES ENTREPRISES DE GARAGE   
1  1180000  COMMISSION PARITAIRE DE L'INDUSTRIE ALIMENTAIRE   
2  1190000     COMMISSION PARITAIRE DU COMMERCE ALIMENTAIRE   
3  1190000     COMMISSION PARITAIRE DU COMMERCE ALIMENTAIRE   
4  1240000          COMMISSION PARITAIRE DE LA CONSTRUCTION   

                                              jcNl  \
0           PARITAIR COMITE VOOR HET GARAGEBEDRIJF   
1       PARITAIR COMITE VOOR DE VOEDINGSNIJVERHEID   
2  PARITAIR COMITE VOOR DE HANDEL IN VOEDINGSWAREN   
3  PARITAIR COMITE VOOR DE HANDEL IN VOEDINGSWAREN   
4             PARITAIR COMITE VOOR HET BOUWBEDRIJF   

                                             titleFr  \
0                               Prime syndicale 2020   
1                               Frais de déplacement   
2                                   Groupes à risque   
3                               Sécurité d'existence   
4  Compensation aux employeurs des sommes dues au...   

                                             titleNl themesFr themesNl  \
0                              Syndicale premie 2020     None     None   
1                                Verplaatsingskosten     None     None   
2                                      Risicogroepen     None     None   
3                                  Bestaanszekerheid     None     None   
4  Verevening aan sommige werkgevers van de bedra...     None     None   

                  signatureDate                  validityDate  \
0  2020-12-14T11:00:00.000+0000  2020-12-31T11:00:00.000+0000   
1  2020-12-22T11:00:00.000+0000                          None   
2  2021-01-27T11:00:00.000+0000  2021-12-31T11:00:00.000+0000   
3  2021-01-27T11:00:00.000+0000  2021-12-31T11:00:00.000+0000   
4  2020-10-08T10:00:00.000+0000                          None   

                    depositDate  ... royalDecreeDate  \
0  2021-02-10T11:00:00.000+0000  ...             NaN   
1  2021-02-12T11:00:00.000+0000  ...             NaN   
2  2021-02-09T11:00:00.000+0000  ...             NaN   
3  2021-02-09T11:00:00.000+0000  ...             NaN   
4  2021-02-10T11:00:00.000+0000  ...             NaN   

            noticeDepositMBDate  publicationRoyalDecreeDate  correctedDate  \
0  2021-03-03T11:00:00.000+0000                         NaN            NaN   
1  2021-03-03T11:00:00.000+0000                         NaN            NaN   
2  2021-03-03T11:00:00.000+0000                         NaN            NaN   
3  2021-03-03T11:00:00.000+0000                         NaN            NaN   
4  2021-03-03T11:00:00.000+0000                         NaN            NaN   

              documentLink documentSize  \
0  112/112-2021-001286.pdf       102 Kb   
1  118/118-2021-001409.pdf         1 Mb   
2  119/119-2021-001241.pdf       316 Kb   
3  119/119-2021-001242.pdf       296 Kb   
4  124/124-2021-001308.pdf       418 Kb   

                                             scopeFr  \
0                                               None   
1                                               None   
2                                               None   
3                                               None   
4  [Entreprises qui occupent moins de 20 travaill...   

                                             scopeNl noScopeFr noScopeNl  
0                                               None      None      None  
1                                               None      None      None  
2                                               None      None      None  
3                                               None      None      None  
4  [Ondernemingen die minder dan 20 werknemers te...      None      None  

[5 rows x 24 columns]

In [7]:
new_df.shape

(79, 24)